In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import cv2
from os.path import join
import os
import random
from functools import partial
from keras.models import Model, load_model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Reshape, concatenate
from keras.optimizers import Nadam
from keras.layers.advanced_activations import ELU
from keras.layers.normalization import BatchNormalization
from keras import backend as K
from keras.backend import binary_crossentropy
import tensorflow as tf

K.set_image_data_format('channels_last')

print(K.image_data_format())
smooth = 1e-12
num_examples = 0

In [ ]:
def applyingclahe(image):
    gray = image
    gray = cv2.cvtColor(gray, cv2.COLOR_BGR2GRAY)
    ret, gray = cv2.threshold(gray, 250, 255,0)
    image, contours, hierarchy = cv2.findContours(gray, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    largest_areas = sorted(contours, key=cv2.contourArea)
    x,y,w,h = cv2.boundingRect(largest_areas[-1])
    mask = np.zeros((512,512),dtype= np.uint8)
    cv2.drawContours(mask, [largest_areas[-1]], 0, (255,255,255,255), -1)
    image = cv2.bitwise_and(orig_images, orig_images, mask=mask)
    roi=image[y-20:y+h+20,x-20:x+w+20]
    roi=cv2.resize(roi,(512,512))
    #plt.imshow(roi)
    #plt.show()

    lab= cv2.cvtColor(roi, cv2.COLOR_BGR2LAB)
    #plt.imshow(lab)
    #plt.show()
    #-----Splitting the LAB image to different channels-------------------------
    l, a, b = cv2.split(lab)
    #plt.imshow(l)
    #plt.show()

    #plt.imshow(a)
    #plt.show()

    #plt.imshow(b)

    #plt.show()
    #-----Applying CLAHE to L-channel-------------------------------------------
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    cl = clahe.apply(l)
    #plt.imshow( cl)
    #plt.show()
    #-----Merge the CLAHE enhanced L-channel with the a and b channel-----------
    limg = cv2.merge((cl,a,b))
    #plt.imshow(limg)
    #plt.show()
    #-----Converting image from LAB Color model to RGB model--------------------
    final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    plt.imshow(final)
    plt.show()
    return final

In [ ]:
orig_images= cv2.imread("/home/ubuntu/research/train/btrain/4117.png")#4117
orig_images = cv2.resize(orig_images, (512, 512))
rows,cols,_ = orig_images.shape

# M = cv2.getRotationMatrix2D((cols/2,rows/2),180,1)
# orig_images = cv2.warpAffine(orig_images,M,(cols,rows))
#plt.imshow(orig_images)
sdd = applyingclahe(orig_images)